In [1]:
import sys

## Analysis

For currency arbitrage problems, graphs are used to represent the data. Currencies are vertices, and exchange rates are edges. Since every currency should be exchangeable, the resulting graph is a complete directed graph.

The objective is to find a cycle in this complete directed graph such that the product of the edge weights within the cycle exceeds 1.01, i.e. 1% arbitrage profit. 

There is one more catch in this specific problem: if arbitrage exists, we need to find the arbitrage that involves with the fewest transactions. Also, even if arbitrage exists but if the number of transactions required exceeds the number of currencies $n$, then we will not consider the solution at all.

Therefore, the key to this question is we want to maximize the product of the edge weights along the cycle and in the meantime, minimize the number of edges that construct the cycle.

Floyd-Warshall algorithm can be used to find all-pairs shortest-path for a directed graph. Since the number of transactions is limited, we can modify the Floyd-Warshall algorithm to find an all-pairs longest-path with at most $n$ edges.

In [2]:
def read_data(file=None):
    """
    To demonstrate the algorithm, a file input was used
    """
    if file is not None:
        sys.stdin = open(file, 'r')
    
    # Need to read multiple graphs
    linebreak = 0  # Record where the adjacency matrix ends
    graphs = []
    
    # Initialize the variables
    adj_matrix = None
    counter = None
    
    for lineidx, line in enumerate(sys.stdin):
        nums = line.strip().split()
        if len(nums) == 1 and lineidx == linebreak:
            if linebreak != 0:
                # Graph adjacency matrix input complete. Store the graph
                graphs.append(adj_matrix)
            
            # Otherwise, initialize the adjacency matrix and start reading the graph.
            graph_size = int(nums[0])
            adj_matrix = [[0. for _ in range(graph_size)] for _ in range(graph_size)]
            
            # Count the rows so that the correct element is filled into the adjacency matrix
            counter = 0
            linebreak = lineidx + graph_size + 1
        
        else:
            # Calculate where to fill in the rate into the adjacency matrix
            indices = [i for i in range(graph_size) if i != counter]
            nums = [float(x) for x in nums]
            for idx, rate in enumerate(nums):
                # Complete the adjacency matrix
                adj_matrix[counter][indices[idx]] = rate
            # Next row
            counter += 1
            
    graphs.append(adj_matrix)
    return graphs

In [12]:
def find_arbitrage(adj_matrix):
    # Number of currencies
    n = len(adj_matrix)
    
    # Account value of a certain trade path.
    # account[step][i][j] --> means the account value after we traded "step" times starting 
    # with currency i and ends in currency j. The starting value of the account is 1 as we are calculating percentage.
    
    # At step 0, only one trade is allowed. Therefore, account[0][i][j] = adj_matrix[i][j] if i != j else 1
    account = [[[0. for _ in range(n)] for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                account[0][i][j] = adj_matrix[i][j]
            else:
                account[0][i][j] = 1
    
    # Predecessor table. 
    # prev[step][i][j] --> means when we traded "step" times and started with currency i and ends in currency j,
    # the final trade is converting currency "prev[step][i][j]" to currency j
    
    # At step 0, only one trade is allowed. Therefore, prev[0][i][j] = i
    prev = [[[None for _ in range(n)] for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            prev[0][i][j] = i
    
    # This list is for recording results.
    output = []
    
    # Now we can run the Floyd-Warshall algorithm
    for tx in range(1, n):
        # Already dealt with the one transaction, so we begin with scenarios with two or more transactions.
        # Floyd-Warshall starts here
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    if account[tx-1][i][k] * adj_matrix[k][j] > account[tx][i][j]:
                        account[tx][i][j] = account[tx-1][i][k] * adj_matrix[k][j]
                        prev[tx][i][j] = k
                        
                        # Speeding things up: early exit
                        # As soon as we found an arbitrage, we stop iterating and print the result.
                        if account[tx][i][i] >= 1.01:
                            # We found a path! Print the path and return true.
                            # The arbitrage ends at currency i, so we append it to the output result first.
                            output.append(str(i + 1))

                            # Now we start iterating through the predecessors.
                            m = tx
                            last_ccy = i
                            while m >= 0:
                                last_ccy = prev[m][i][last_ccy]
                                m -= 1
                                output.append(str(last_ccy + 1))

                            # The path is inversed as we appended the destination currency first.
                            print(" ".join(output[::-1]))
                            return
            
    # No path exists. Print the error message.
    print("no arbitrage sequence exists")

Example test cases:

In [13]:
data = read_data("input.txt")

In [14]:
find_arbitrage(data[0])

1 2 1


In [15]:
find_arbitrage(data[1])

1 4 3 1


In [7]:
find_arbitrage(data[2])

no arbitrage sequence exists


While the answer did not exactly match the ones in problem description, the number of trades is correct. This implies that the answer is not unique. Even with the same number of trades, multiple paths leading to arbitrage exist. 

In [ ]:
"""
This section is for OJ submission.
"""

if __name__ == "__main__":
    graphs = read_data()
    for g in graphs:
        find_arbitrage(g)